# TF-Enrichment

In [ ]:

def create_geneList_file(geneList):
    with open('CHEA/geneList.txt','w') as file:
        for g in geneList:
            file.write(g+'\n')

#create_DF_from_KEAoutput  
def create_DF_from_KEAoutput(expt, finalDF):
    KEAout = pd.read_csv('CHEA/CHEA_output.csv', header=None, index_col=False)
    KEAout.head()
    KEAout.columns = ['Kinase','number of substrates in the input gene-list', 'number of genes that are substrates of the kinase',\
                      'fraction of genes that are substrates compared to total number of genes in gene-list',\
                      'fraction of genes that are substrates compared to total number of genes in background',\
                      'difference between the background fraction and the substrate-list fraction',\
                      'pvalue', 'ztest_rank', 'combined_score','substrates']
    KEAout.index = KEAout['Kinase']
    KEA_sort = KEAout.sort_values(by='pvalue')
    KEA_sort[expt] = range(0,len(KEA_sort))
    newDF = KEA_sort[['Kinase',expt]]
    finalDF = finalDF.merge(newDF, on='Kinase', how='outer')
    return finalDF

def return_negLog(expt, finalDF):
    KEAout = pd.read_csv('KEA/KEA_output.csv', header=None, index_col=False)
    KEAout.columns = ['Kinase','number of substrates in the input gene-list', 'number of genes that are substrates of the kinase',\
                      'fraction of genes that are substrates compared to total number of genes in gene-list',\
                      'fraction of genes that are substrates compared to total number of genes in background',\
                      'difference between the background fraction and the substrate-list fraction',\
                      'pvalue', 'ztest_rank', 'combined_score','substrates']
    KEAout.index = KEAout['Kinase']
    KEAout.head()
    # Create -log(pvalue)
    KEAout['-log(pvalue)'] = -np.log(pd.to_numeric(KEAout['pvalue']))
    KEA_sort = KEAout.sort_values(by=['-log(pvalue)'])[['Kinase','-log(pvalue)']] #** Select values to put in newDF
    KEA_sort.columns = ['Kinase', expt]
    finalDF = finalDF.merge(KEA_sort, on='Kinase', how='outer')
    return finalDF
    
    

In [ ]:
# CHEA-specific
import pandas as pd
import numpy as np
import os 
from time import sleep
   
def run_CHEA_old(inputGMT, KEA_summary_file, replaceNAs=True):
    with open(inputGMT) as file:
        input_GMT = file.readlines()
    #input_GMT = input_GMT[0:10]
    finalDF=pd.DataFrame(columns=['Kinase'])
    for line in input_GMT:
        # Delete old files
        try:
            os.remove('CHEA/CHEA_output.csv')
            os.remove('CHEA/geneList.txt')
        except:
            print("No files to delete")
        while os.path.exists('KEA/geneList.txt') or os.path.exists('CHEA/CHEA_output.csv'):
            sleep(.5)
        
        # Create gene list
        lineSp = line.split('\t')
        expt = lineSp[0]
        genes = [x.strip(',1.0') for x in lineSp[2:-1]]
        print("Processing: "+expt)
        # Create gene list txt file
        print(expt+': Creating genList file')
        create_geneList_file(genes)
        print('Waiting for KEA_output')
        while not os.path.exists('KEA/geneList.txt'):
            sleep(.5) 
        # Run KEA command line
        # result = subprocess.run(['/Library/Java/JavaVirtualMachines/1.6.0.jdk/Contents/Home/bin/java','-jar',\
        #                          'KEA/KEA-1.5-SNAPSHOT-jar-with-dependencies.jar','KEA/UberKeaFile.csv',\
        #                          'KEA/geneList.txt KEA/KEA_output.csv'] )
        # result.stdout.decode('utf-8')
        print('Running KEA')
        os.system('/Library/Java/JavaVirtualMachines/1.6.0.jdk/Contents/Home/bin/java '+\
                  ' -jar'+' KEA/KEA-1.5-SNAPSHOT-jar-with-dependencies.jar'+' ../../X2K_Databases/KINASE/KEA_2018/KEA2018_KINASES.csv'+\
                  ' KEA/geneList.txt'+ ' KEA/KEA_output.csv')
        # Sleep until the file is ready
        print('Waiting for KEA_output')
        while not os.path.exists('KEA/KEA_output.csv'):
            sleep(.5)
            
        # Read in KEA output and process
        print(expt+' : Creating dataframe')
        
        finalDF = return_negLog(expt, finalDF) #*** Control whether you what values are in final DF
        if replaceNAs==True:
            finalDF = replace_NAs_with_random_rank(finalDF)
        
    finalDF.to_csv(KEA_summary_file, sep='\t', header=True, index=None, na_rep='NA')
    return finalDF

run_CHEA_old(inputGMT='GEO.txt', )


In [ ]:
CHEA_UP = run_CHEA_old(inputGMT='Kinase_Perturbations_from_GEO_up.txt', KEA_summary_file='CHEA_output_NAsfilled_UP.txt', replaceNAs=True)
CHEA_DN = run_CHEA_old(inputGMT='Kinase_Perturbations_from_GEO_down.txt', KEA_summary_file='CHEA_output_NAsfilled_DN.txt', replaceNAs=True)
